In [1]:
import DataStructures: OrderedDict
using RigidBodyDynamics
using StaticArrays
using CoordinateTransformations
using Interact
using LCMGL
using ForwardDiff
using DataFrames, DataArrays

  likely near /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/Interact/src/IJulia/setup.jl:116
  likely near /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/Interact/src/IJulia/setup.jl:116


In [2]:
include("arms.jl")
include("depth_sensors.jl")

DepthSensors

In [3]:
function beanbag()
    limbs = OrderedDict{RigidBody, Arms.Limb}()
    
    mechanism = Mechanism(RigidBody{Float64}("world"))
    parent = root_body(mechanism)
    
    joint = Joint("joint1", QuaternionFloating())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    surface_points = Vector{Point3D}()
    skeleton_points = [Point3D(body.frame, @SVector [0, 0, 0])]
    for axis = 1:3
        for s = [-1; 1]
            x = [0., 0, 0]
            x[axis] = s
            push!(surface_points, Point3D(body.frame, SVector(x...)))
        end
    end
    limbs[body] = Arms.Limb(surface_points, skeleton_points)
    
    Arms.Model(mechanism, limbs)
end

beanbag (generic function with 1 method)

In [4]:
model = beanbag()

Arms.Model(Vertex: world (root)
  Vertex: body1, Edge: joint1,DataStructures.OrderedDict{RigidBodyDynamics.RigidBody,Arms.Limb}(RigidBody: "body1"=>Arms.Limb(RigidBodyDynamics.Point3D[Point3D in "body1": [-1.0,0.0,0.0],Point3D in "body1": [1.0,0.0,0.0],Point3D in "body1": [0.0,-1.0,0.0],Point3D in "body1": [0.0,1.0,0.0],Point3D in "body1": [0.0,0.0,-1.0],Point3D in "body1": [0.0,0.0,1.0]],RigidBodyDynamics.Point3D[Point3D in "body1": [0,0,0]])))

In [5]:
state = Arms.ModelState(model)
Arms.draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f43946392d0>))

In [6]:
state.limb_deformations[1][1].frame

CartesianFrame3D: "body1"

In [7]:
joint_angles, deformations = Arms.zero_configuration(model)
state = Arms.ModelState(model, joint_angles, deformations)
@manipulate for x = linspace(-1, 1, 51)
    state.limb_deformations[1][1] = FreeVector3D(model.mechanism.toposortedTree[2].vertexData.frame, @SVector [x, 0, 0])
    Arms.draw(model, state)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),".3f",true)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f4394639300>))

In [8]:
# Construct the sensor and draw its view rays for debugging
sensor = DepthSensors.Kinect(80,80);
camera_origin = @SVector [0., 0, 8]
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi, 1, 0, 0)))
# camera_origin = tformtranslate([0;0;8]) * tformrotate([pi;0;0])
LCMGLClient("sensor_rays") do lcmgl
    LCMGL.color(lcmgl, 0, 1, 0)
    begin_mode(lcmgl, LCMGL.LINES)
    for ray in DepthSensors.rays_in_world(sensor, camera_tform)
        vertex(lcmgl, camera_origin...)
        vertex(lcmgl, (camera_origin + ray)...)
    end
    end_mode(lcmgl)
    switch_buffer(lcmgl)
end

In [9]:
function raycast(model::Arms.Model, state::Arms.ModelState)
    Arms.draw(model, state, true)
    skin = Arms.skin(model, state)
    
    points = DepthSensors.raycast_points(skin, sensor, camera_tform)
    LCMGLClient("raycast") do lcmgl
        LCMGL.color(lcmgl, 0, 1, 0)
        point_size(lcmgl, 5)
        begin_mode(lcmgl, LCMGL.POINTS)
        for point in points
            vertex(lcmgl, convert(Vector, point)...)
        end
        end_mode(lcmgl)
        switch_buffer(lcmgl)
    end
    points
end

raycast (generic function with 1 method)

In [10]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = 2*rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = rand(SVector{3, Float64}) - 0.5
    end
end
state = Arms.ModelState(model, joint_angles, deformations)
skin = Arms.skin(model, state)

(::InterpolatingSurface) (generic function with 1 method)

In [11]:
function flatten(state::Arms.ModelState)
    data = configuration_vector(state.mechanism_state)
    for limb_deformation in state.limb_deformations
        for d in limb_deformation
            for x in d.v
                push!(data, x)
            end
        end
    end
    data
end

function unflatten(model::Arms.Model, state::Arms.ModelState, data)
    nq = num_positions(state.mechanism_state)
    set_configuration!(state.mechanism_state, data[1:nq])
    
    data_index = nq + 1;
    for (i, (body, limb)) in enumerate(model.limbs)
        limb_deformation = state.limb_deformations[i]
        for (j, deformation) in enumerate(limb_deformation)
            state.limb_deformations[i][j] = RigidBodyDynamics.FreeVector3D(body.frame, @SVector [data[data_index + i] for i in 0:2])
            data_index += 3
        end
    end
    @assert data_index == length(data) + 1
    state
end

unflatten (generic function with 1 method)

In [12]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = 0.5*(rand(SVector{3, Float64}) - 0.5)
    end
end

true_state = Arms.ModelState(model, joint_angles, deformations)
x_true = flatten(true_state)



sensed_points = raycast(model, true_state)

x_ad = []
state_ad = []

cost = x -> begin
    push!(x_ad, x)
    state = Arms.ModelState(model, eltype(x), eltype(x))
    unflatten(model, state, x)
    push!(state_ad, state)
    Arms.draw(model, state, false)
    skin = Arms.skin(model, state)
    raycast_cost = sum(point -> skin(point)^2, sensed_points)
    deformation_cost = zero(eltype(x))
    for ld in state.limb_deformations
        for d in ld
            deformation_cost += sum(d.v .^2)
        end
    end
    return raycast_cost + 10*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
cost_and_gradient = x -> begin
    out = GradientResult(x)
    ForwardDiff.gradient!(out, cost, x)
    ForwardDiff.value(out), ForwardDiff.gradient(out)
end

trials = []

function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Arms.ModelState(model))

        # Now we use gradient descent to recover the joint angles
        for i = 1:500
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci, gi = cost_and_gradient(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end

            # Descend the gradient
            x_estimated -= 0.002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [13]:
test_gradient_descent()
state = unflatten(model, Arms.ModelState(model), trials[1][:x][end])
Arms.draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 6040xFixedSizeArrays.Point{3,Float64},     faces: 12076xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f4394639450>))